In [1]:
# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
# make variables display whenever they are on their own line (not just the last line of a cell)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [21]:
import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from utils import *

In [28]:
tf.reset_default_graph()
sess = tf.Session()
input_ph = tf.placeholder('float', (None, 224, 224, 3))

In [48]:
dataset = make_precomputed_dataset('../val2017', 'conv1')

In [49]:
sess.run(dataset.make_one_shot_iterator().get_next())

b'../val2017\\conv1\\000000000139.npy'


UnknownError: FileNotFoundError: [Errno 2] No such file or directory: "b'../val2017\\\\conv1\\\\000000000139.npy'"
	 [[Node: PyFunc = PyFunc[Tin=[DT_STRING], Tout=[DT_FLOAT], token="pyfunc_7"](arg1)]]
	 [[Node: IteratorGetNext_13 = IteratorGetNext[output_shapes=[[224,224,3], <unknown>], output_types=[DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](OneShotIterator_13)]]

In [ ]:
make_encoder(input_ph, sess)
reconstructed_image = make_decoder(vgg[1]['vgg_19/conv2/conv2_1'], [(3, 64), 'upsample', (3, 64), (3, 3)], sess)

In [ ]:
loss = tf.sqrt(tf.reduce_mean(tf.square(input_ph - reconstructed_image)))

In [ ]:
train(loss, dataset, input_ph, sess, num_epochs=10, lr=1e-4)

In [ ]:
it = dataset.make_one_shot_iterator()

In [ ]:
img = sess.run(tf.cast(it.get_next(), tf.uint8))

In [ ]:
plt.imshow(img)

In [ ]:
decoded_img = sess.run(reconstructed_image, feed_dict={input_ph: img[np.newaxis,:,:,:]})

In [ ]:
decoded_img = np.squeeze(decoded_img)

In [ ]:
decoded_img[decoded_img > 255] = 255

In [ ]:
plt.imshow(decoded_img.astype(np.uint8))

In [ ]:
writer = tf.summary.FileWriter('summaries', sess.graph)

In [19]:
compute_features('../val2017', 'conv1.tfrecord', out_layer_name='vgg_19/conv1/conv1_1', num_images=200)

INFO:tensorflow:Restoring parameters from vgg_19.ckpt
